<a href="https://colab.research.google.com/github/DarshK01/Portfolio/blob/main/AI_Research_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==============================================================================
# 1. INSTALLATION - Installs all necessary and compatible libraries
# ==============================================================================
# ==============================================================================
# Step 0: Set the Memory Allocator Configuration (The Fix)
# ==============================================================================
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print("✅ PYTORCH_CUDA_ALLOC_CONF set to 'expandable_segments:True'")

# ==============================================================================
# Step 1: Install all necessary and compatible libraries
# ==============================================================================
print("🚀 Installing libraries...")
!pip install -q transformers==4.41.2 peft==0.11.1 accelerate==0.30.1 bitsandbytes>=0.43.2 trl==0.8.6 triton

print("✅ All libraries installed successfully!")




✅ PYTORCH_CUDA_ALLOC_CONF set to 'expandable_segments:True'
🚀 Installing libraries...
✅ All libraries installed successfully!


In [4]:
# ==============================================================================
# 2. LOGIN TO HUGGING FACE
# ==============================================================================

from huggingface_hub import notebook_login

print("Please enter your Hugging Face access token with 'write' permissions.")
notebook_login()

Please enter your Hugging Face access token with 'write' permissions.


In [9]:
# Step 3: Loading Model and Tokenizer

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print("🚀 Loading Model and Tokenizer...")
model_name = "microsoft/Phi-3-mini-4k-instruct"

# QLoRA configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fixes a warning message during training

print(f"✅ Model '{model_name}' loaded successfully!")

🚀 Loading Model and Tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


✅ Model 'microsoft/Phi-3-mini-4k-instruct' loaded successfully!


In [18]:
# Step 4: Preparing the dataset

from datasets import load_dataset

print("\n🚀 Step 4: Preparing the dataset...")
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
dataset = load_dataset(dataset_name, split="train")

# Rename the 'output' column to 'completion' to match SFTTrainer's expectation
dataset = dataset.rename_column("output", "completion")

print("✅ Dataset prepared successfully!")


🚀 Step 4: Preparing the dataset...
✅ Dataset prepared successfully!


In [19]:
print("\n🚀 Step 5: Configuring training...")

from peft import LoraConfig
from transformers import TrainingArguments

# LoRA config (no changes here)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# --- MODIFICATION IS HERE ---
training_args = TrainingArguments(
    output_dir="./phi3-python-assistant",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    optim="adamw_8bit",  # Use the 8-bit AdamW optimizer to save memory
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    save_strategy="epoch",
    push_to_hub=True,
    hub_model_id="DarshaniKare/Phi3-mini-Python-Assistant",
)

print("✅ Training configuration is set with 8-bit optimizer for maximum memory saving.")


🚀 Step 5: Configuring training...
✅ Training configuration is set with 8-bit optimizer for maximum memory saving.


In [25]:
# Step 6: Initializing Trainer

from trl import SFTTrainer

print("🚀 Step 6: Initializing Trainer...")

# Create the trainer with the final memory optimization
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="prompt",
    max_seq_length=512,
)

print("🔥 Starting the training process...")
trainer.train()
print("✅ Training complete!")

🚀 Step 6: Initializing Trainer...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:479: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


🔥 Starting the training process...


wandb: Currently logged in as: darshanikare (darshanikare-usha-mittal-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.751200
20,0.550700
30,0.522500
40,0.467300
50,0.454300
60,0.470400
70,0.490800
80,0.474000
90,0.450700
100,0.478300


Step,Training Loss
10,0.751200
20,0.550700
30,0.522500
40,0.467300
50,0.454300
60,0.470400
70,0.490800
80,0.474000
90,0.450700
100,0.478300


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Training complete!


In [26]:
# ==============================================================================
# 7. INFERENCE (TESTING THE MODEL)
# ==============================================================================
print("\n🚀 Step 7: Testing the fine-tuned model...")
from transformers import pipeline

prompt = "Write a Python function that finds the longest word in a sentence."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")

print("\n--- MODEL RESPONSE ---")
print(result[0]['generated_text'])
print("--------------------")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🚀 Step 7: Testing the fine-tuned model...

--- MODEL RESPONSE ---
[INST] Write a Python function that finds the longest word in a sentence. [/INST] def longest_word(sentence):
    words = sentence.split()
    longest_word = ""
    for word in words:
        if len(word) > len(longest_word):
            longest_word = word
    return longest_word

sentence = "This is a sample sentence"
longest_word = longest_word(sentence)
print(longest_word) # Output: "sentence"

--------------------
